In [1]:
!apt update && apt install -y curl git

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 264 kB in 3s (103 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
47 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' a

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!ollama pull llama3


Error: could not connect to ollama app, is it running?


In [4]:
#  !pkill -f ngrok
# !fuser -k 8501/tcp  to skill the ngrock port

In [5]:
OLLAMA_API_URL = "http://127.0.0.1:11434"
!nohup ollama serve > /dev/null 2>&1 &

In [6]:
!pip install pyngrok

!pip install streamlit PyPDF2 requests python-docx pymupdf crewai


In [7]:
from pyngrok import ngrok

# Replace with your ngrok auth token
NGROK_AUTH_TOKEN = "2WG8G2YZNV9jofWBKu5B2E6hA3Q_719vVw9PLU1SwQLwCtcNd"

# Set authentication token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok Auth Token Set!")

public_url = ngrok.connect(8501).public_url
print(f"🚀 Streamlit App is running at: {public_url}")


✅ ngrok Auth Token Set!
🚀 Streamlit App is running at: https://be0d-34-87-119-7.ngrok-free.app


In [8]:
import requests

OLLAMA_API_URL = "http://127.0.0.1:11434/api/generate"

headers = {"Content-Type": "application/json"}
payload = {
    "model": "llama3",
    "prompt": "What is the capital of France?",
    "stream": False
}

response = requests.post(OLLAMA_API_URL, json=payload, headers=headers)

if response.status_code == 200:
    print("✅ Response from Ollama:", response.json()["response"])
else:
    print(f"❌ Error from Ollama: {response.status_code}, {response.text}")


✅ Response from Ollama: The capital of France is Paris.


In [9]:
%%writefile app.py
import streamlit as st
import asyncio
asyncio.set_event_loop(asyncio.new_event_loop())

import fitz  # PyMuPDF for PDF processing
import requests
import tempfile
import os
import torch
from sentence_transformers import SentenceTransformer, util
from crewai import Agent, Task, Crew

# Load the Sentence-BERT model for bi-encoder reranking
bi_encoder = SentenceTransformer("all-MiniLM-L6-v2")  # Small and efficient SBERT model

# Ollama API URL for Llama 3
OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

# Streamlit Page Config
st.set_page_config(page_title="Chat with PDF (Llama 3)", layout="wide")
st.title("📄 Chat with Your PDF (Llama 3)")
st.caption("Upload a PDF and ask questions about its content!")

# Step 1: PDF Processing Class
class PDFProcessor:
    """Extracts text from a PDF file using PyMuPDF"""
    def extract_text(self, pdf_path):
        doc = fitz.open(pdf_path)  # Open PDF
        text = "\n".join([page.get_text() for page in doc])  # Extract text from each page
        return text.strip()  # Remove extra whitespace

# Instantiate the PDF processor
pdf_processor = PDFProcessor()

# Step 2: Define CrewAI Agents
pdf_agent = Agent(
    role="PDF Processor",
    goal="Extract text from PDF documents",
    backstory="A digital assistant specialized in reading and extracting content from PDF files.",
    verbose=True,
    allow_delegation=False
)

qa_agent = Agent(
    role="AI Question Answerer",
    goal="Answer questions based on extracted text using Llama 3",
    backstory="A knowledgeable AI capable of understanding documents and providing insightful answers.",
    verbose=True
)

# Step 3: Define Tasks for CrewAI
pdf_extraction_task = Task(
    description="Extract text from a given PDF file and provide structured content.",
    agent=pdf_agent,
    expected_output="A structured text output of the PDF content."
)

qa_task = Task(
    description="Use the extracted PDF text to answer user questions accurately.",
    agent=qa_agent,
    expected_output="A well-formulated answer to the given question."
)

# Step 4: Crew Setup (Manages the workflow)
crew = Crew(
    agents=[pdf_agent, qa_agent],
    tasks=[pdf_extraction_task, qa_task]
)

# Step 5: Function to Find Relevant Chunk Using SBERT
def find_relevant_chunk(text, question, window_size=3000, overlap=500):
    """
    Uses Sentence-BERT (bi-encoder) to find the most relevant chunk of text.
    - Splits text into overlapping chunks
    - Encodes chunks & query with SBERT
    - Uses cosine similarity to find the best match
    """

    # Step 5.1: Create overlapping chunks
    chunks = []
    step = window_size - overlap  # Define step size for the sliding window

    for i in range(0, len(text), step):
        chunks.append(text[i : i + window_size])  # Append overlapping chunks

    # Step 5.2: Encode the question using SBERT
    query_embedding = bi_encoder.encode(question, convert_to_tensor=True)

    # Step 5.3: Encode all chunks
    chunk_embeddings = bi_encoder.encode(chunks, convert_to_tensor=True)

    # Step 5.4: Compute similarity scores between query and each chunk
    similarity_scores = util.pytorch_cos_sim(query_embedding, chunk_embeddings)[0]

    # Step 5.5: Find the chunk with the highest similarity score
    best_idx = torch.argmax(similarity_scores).item()
    best_chunk = chunks[best_idx]  # Select the most relevant chunk

    return best_chunk  # Return the best chunk to be sent to the LLM

# Step 6: Streamlit UI for PDF Upload
uploaded_file = st.file_uploader("📂 Upload a PDF", type=["pdf"])

if uploaded_file:
    st.success(f"✅ Uploaded: {uploaded_file.name}")

    # Step 7: Save uploaded PDF to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(uploaded_file.read())  # Save PDF content
        temp_pdf_path = temp_file.name  # Get temp file path

    # Step 8: Extract text from the uploaded PDF
    pdf_text = pdf_processor.extract_text(temp_pdf_path)

    # Delete temp file after extracting text
    os.remove(temp_pdf_path)

    if pdf_text:
        st.info("📄 PDF text extracted successfully!")

        # Step 9: Allow users to view extracted text (first 1000 characters)
        with st.expander("🔍 View Extracted Text"):
            st.text_area("PDF Content", pdf_text[:1000] + "..." if len(pdf_text) > 1000 else pdf_text, height=200)

        # Step 10: User enters a question
        question = st.text_input("💡 Ask a question about the document")

        if question:
            st.info("🤖 Finding the most relevant section using SBERT...")

            # Step 11: Get the most relevant chunk using SBERT reranking
            relevant_chunk = find_relevant_chunk(pdf_text, question)

            st.info("🤖 Sending the relevant section to Llama 3...")

            # Step 12: Send the relevant chunk to Llama 3
            ollama_payload = {
                "model": "llama3",
                "prompt": f"Context: {relevant_chunk}\n\nQuestion: {question}\n\nAnswer:",
                "stream": False  # Set to True if you want to stream responses
            }
            response = requests.post(OLLAMA_URL, json=ollama_payload)

            # Step 13: Handle the response
            if response.status_code == 200:
                result = response.json().get("response", "⚠️ No response from Llama 3.")
                st.subheader("📌 Answer:")
                st.write(result)
            else:
                st.error("⚠️ Failed to get response from Llama 3. Is Ollama running?")


Overwriting app.py


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.119.7:8501

2025-03-03 18:15:41.514185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741025741.811654   10560 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741025741.893967   10560 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 18:15:42.495768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild